In [25]:
from textblob import TextBlob
import pandas as pd
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
import altair as alt
alt.data_transformers.disable_max_rows()
import numpy as np

In [8]:
# kw_model = KeyBERT()
# kw_model.extract_keywords(docs=[txt], use_maxsum=True, vectorizer=KeyphraseCountVectorizer(), stop_words="english", top_n=20)

In [43]:
df = pd.read_csv(
    "data/enron/raw.csv",
    usecols=["Message-ID", "Date", "content"],
    parse_dates=["Date"],
).rename(columns={"Message-ID": "category", "Date": "date"})
df["category"] = df["category"].str[1:23].str.replace(".", "")
df


,category,date,content
0,187829811075855378110,2001-05-14 23:39:00,Here is our forecast
1,154649861075855378456,2001-05-04 20:51:00,Traveling to have a business meeting takes the...
2,242162401075855687451,2000-10-18 10:00:00,test successful. way to go!!!
3,135058661075863688222,2000-10-23 13:13:00,"Randy, Can you send me a schedule of the salar..."
4,309229491075863688243,2000-08-31 12:07:00,Let's shoot for Tuesday at 11:45.
...,...,...,...
517396,268079481075842029936,2001-11-28 21:30:11,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,258358611075842029959,2001-11-28 20:47:48,Some of my position is with the Alberta Term b...
517398,289798671075842029988,2001-11-28 15:20:00,"2 -----Original Message----- From: Doucet, Daw..."
517399,220525561075842030013,2001-11-27 19:52:45,Analyst Rank Stephane Brodeur 1 Chad Clark 1 I...


In [49]:
df["year"] = df["date"].dt.strftime("%Y-%m")
disp = df.groupby("year", as_index=False)['content'].count()
alt.Chart(disp).mark_bar(color="purple").encode(x="year:O", y="content").properties(width=900)

alt.Chart(...)

: 

In [28]:
df = df[df["date"].dt.year.isin(range(1997, 2005))]
df[["category", "date"]].to_csv("data/enron/metadata.csv", index=False)
df = df.set_index(["category", "date"])
df["content"] = (
    df["content"]
    .replace("‘", "'")
    .replace("’", "'")
    .replace("“", '"')
    .replace("”", '"')
)

In [32]:
pd.read_csv("results/enron/dvr.csv").head(20)

,element_code,element,global_weight
0,621888,ect,0.024199
1,660150,enron,0.018691
2,1309014,pm,0.010263
3,1598346,subject,0.008128
4,1307315,please,0.006271
5,747168,forwarded,0.005793
6,1651950,thanks,0.005455
7,1395143,re,0.004406
8,1066043,mark,0.004364
9,543905,dc,0.003800


In [29]:
dates = [x for x in pd.date_range(end=pd.datetime.today(), periods=1800)]
counts = [x for x in np.random.randint(0, 10000, size=1800)]
df = pd.DataFrame({'dates': dates, 'counts': counts}).set_index('dates')

In [13]:
l = []
for i, it in enumerate(df.iterrows()):
    if i <= 500000:
        continue
    index, row = it
    npc = TextBlob(str(row["content"])).np_counts
    if npc == {}:
        continue
    adf = pd.DataFrame.from_records(list(npc.items()), columns=["element", "frequency_in_category"])
    adf["category"] = index[0]
    l.append(adf)
concated = pd.concat(l)
concated.to_csv(f"data/enron/np_freq/{i}.csv")
l = []